# Combine individual z-layers and channels to multichannel z-stacks (Setup)

This is a setup notebook for adjusting parameters for [run_images_to_stack.ipynb](run_images_to_stack.ipynb).

Run this notebook cell-by-cell and follow the instructions.

## Requirements

- A folder with images to convert

Individual channels and z-layers must be clearly tagged in the filename, and the naming pattern must be consistent across the dataset. The naming pattern should follow the following scheme: 

`[SAMPLENAME][Z_CODE][Z#][CHANNEL_CODE][CHANNEL#].tif` or `[SAMPLENAME][CHANNEL_CODE][CHANNEL#][Z_CODE][Z#].tif`

**Example**: `"condition1/my_sample_position-3_Z00_C01.tif”`

`[SAMPLENAME]` may include subfolders but must be the same for all files that will be combined.

<hr style="height:2px;">

## Config

<hr style="height:2px;">

### The following code imports and declares functions used for the processing:

In [1]:
#################################
#  Don't modify the code below  #
#################################

import os
import numpy as np
import intake_io
from am_utils.utils import walk_dir
from punctatools.lib.convert import load_random_dataset, check_metadata, get_number_of_stacks
from punctatools.lib.utils import save_parameters

2022-04-06 16:05:49,811 [INFO] WRITING LOG OUTPUT TO /home/amedyukh/.cellpose/run.log


<hr style="height:2px;">

## Set up the data and parameters

<hr style="height:2px;">

### Please provide data paths:

`input_dir`: folder with images to be converted

`output_dir`: folder to save results

In [2]:
input_dir = "../example_data/slices"
output_dir = "../test_output/stacks"

### The following code lists all image files in the input directory:

In [3]:
#################################
#  Don't modify the code below  #
#################################

input_dir = os.path.realpath(input_dir)

samples = walk_dir(input_dir)

print(f'{len(samples)} images were found:')
print(np.array(samples))

30 images were found:
['/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z20_C0.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z20_C1.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z20_C2.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z21_C0.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z21_C1.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z21_C2.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z22_C0.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/sample1_position1_Z22_C1.tif'
 '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices/

### Please specify codes for channel and z-position:

Specify the sequence of characters that precedes the channel and z-position numbering, including the separator (e.g. "_") that precedes the channel code.

For example, if the image name is "my_experiment_position1_C0_z001.tif", the codes should be as follows:

`channel_code` = "_C"

`z_position_code` = "_z"


In [4]:
channel_code = "_C"
z_position_code = "_Z"

### The following code returns the number of stacks that were found

If executing this cell results in an error, make sure the channel and z-position codes are provided correctly. Note the the codes are case-sensitive

In [5]:
#################################
#  Don't modify the code below  #
#################################

n = get_number_of_stacks(input_dir, channel_code=channel_code, z_position_code=z_position_code)
n

2

### The following code checks image metadata

In [6]:
#################################
#  Don't modify the code below  #
#################################

dataset, spacing = check_metadata(input_dir, channel_code=channel_code, z_position_code=z_position_code)
print('The following voxel size was detected:')
if 'z' in dataset.dims:
    dims = ['z', 'y', 'x']
else:
    dims = ['y', 'x']
for s, c in zip(spacing, dims):
    print(rf"{c}: {s}")

The following voxel size was detected:
z: None
y: 0.11000000110000001
x: 0.11000000110000001


<hr style="height:0.5px;">

### Please specify correct voxel size 

Specify the correct voxel size (pixel size and z-stepping) in microns if the above values are wrong.

Keep `None`, if the value loaded from the metadata is correct.

In [7]:
z = 0.2
y = None
x = None

### The following code loads a random stack with correct voxel size

In [8]:
#################################
#  Don't modify the code below  #
#################################

if 'z' in dataset.dims:
    spacing = [z, y, x]
else:
    spacing = [y, x]
dataset = load_random_dataset(input_dir, channel_code=channel_code, 
                              z_position_code=z_position_code, spacing=spacing)
dataset

<xarray.Dataset>
Dimensions:  (y: 326, x: 326, z: 5, c: 3)
Coordinates:
  * y        (y) float64 0.0 0.11 0.22 0.33 0.44 ... 35.42 35.53 35.64 35.75
  * x        (x) float64 0.0 0.11 0.22 0.33 0.44 ... 35.42 35.53 35.64 35.75
  * z        (z) float64 0.0 0.2 0.4 0.6 0.8
Dimensions without coordinates: c
Data variables:
    image    (c, z, y, x) uint16 99 102 99 102 103 99 ... 101 102 96 100 104 103
Attributes:
    metadata:  {'spacing_units': {'x': 'µm', 'y': 'µm', 'z': 'µm'}}

<hr style="height:2px;">

## Run the full processing

The parameters set up in this notebook will be used to convert images in batch.

<hr style="height:2px;">

### Please provide the file name to save the parameters

If only a filename is provided, the parameter file will be saved in the same directory with this notebook. Provide the full path to save the parameters to a different directory.

Default is `parameters.json`

In [9]:
parameter_file = 'parameters.json'

### Please specify the number of processes to run in parallel


`n_jobs`: number of processes to run in parallel. Set according to your workstation resources. Decrease if the processing runs out of memory. (Default is 8)

In [10]:
n_jobs=8

### The following code saves the parameters

To run the full processing, run the notebook [run_images_to_stack.ipynb](run_images_to_stack.ipynb) with the parameter file specified here. 

Alternatively run the [run_images_to_stack.py](../scripts/run_images_to_stack.py) script:

``python run_images_to_stack.py -p <parameter_file>``

In [11]:
#################################
#  Don't modify the code below  #
#################################

spacing = list(intake_io.get_spacing(dataset))

params = dict(
    parameter_file_path=os.path.realpath(parameter_file),
    raw_dir=os.path.realpath(input_dir),
    converted_data_dir=os.path.realpath(output_dir),
    spacing=spacing,
    channel_code=channel_code,
    z_position_code=z_position_code,
    n_jobs=n_jobs
)
params = save_parameters(params, parameter_file)
    
params

{'parameter_file_path': '/research/sharedresources/cbi/common/Anna/codes/punctatools/notebooks/parameters.json',
 'raw_dir': '/research/sharedresources/cbi/common/Anna/codes/punctatools/example_data/slices',
 'converted_data_dir': '/research/sharedresources/cbi/common/Anna/codes/punctatools/test_output/stacks',
 'spacing': [0.2, 0.11000000110000001, 0.11000000110000001],
 'channel_code': '_C',
 'z_position_code': '_Z',
 'n_jobs': 8}